<a href="https://colab.research.google.com/github/emely3h/Geospatial_ML/blob/main/metrics_bug.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Problem with multiclass intersection over union calculation

### 0. Prepare Colab, Define Constants

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#! ls
%cd drive/MyDrive/MachineLearning/
#! git clone https://github.com/emely3h/Geospatial_ML.git
%cd Geospatial_ML
! ls
! git pull
! git checkout feat/experiment-4

/content/drive/.shortcut-targets-by-id/15HUD3sGdfvxy5Y_bjvuXgrzwxt7TzRfm/MachineLearning
/content/drive/.shortcut-targets-by-id/15HUD3sGdfvxy5Y_bjvuXgrzwxt7TzRfm/MachineLearning/Geospatial_ML
data_exploration  evaluation   models	    prepare_data    README.md
docs		  experiments  poetry.lock  pyproject.toml  requirements.txt
Already up to date.
Already on 'feat/experiment-4'
Your branch is up to date with 'origin/feat/experiment-4'.


In [ ]:
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from keras.models import Model
from keras.layers import (
    Input,
    Conv2D,
    MaxPooling2D,
    concatenate,
    Conv2DTranspose,
    Dropout,
    UpSampling2D
)
from keras.losses import categorical_crossentropy
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
import pickle
from keras.utils import Sequence
from datetime import datetime
from models.unet_model import unet_2d
from data_exploration.mask_stats import Mask_Stats
from tensorflow.keras.models import load_model
from evaluation.evaluation_metrics_total import EvaluationMetricsTotal
from models.helpers import save_metrics, predictions_for_models
from evaluation.helpers import plot_loss_acc, load_metrics_into_df, calculate_save_metrics
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical

In [ ]:
total_tiles = 11121
train_tiles = 6672
test_val_tiles = 2224
data_path = "../data_colab/256_256"
experiment = "experiment_6"
batch_size = 32
tile_size = 256
step_size = 25

### 1. Create Data Generators

In [ ]:
train_split_x = np.memmap(os.path.join(data_path, "train_split_x.npy"), mode="r", shape=(train_tiles, 256, 256, 5),
                          dtype=np.uint8)
train_split_y = np.memmap(os.path.join(data_path, "train_split_y.npy"), mode="r", shape=(train_tiles, 256, 256),
                          dtype=np.uint8)
val_split_x = np.memmap(os.path.join(data_path, "val_split_x.npy"), mode="r", shape=(test_val_tiles, 256, 256, 5),
                        dtype=np.uint8)
val_split_y = np.memmap(os.path.join(data_path, "val_split_y.npy"), mode="r", shape=(test_val_tiles, 256, 256),
                        dtype=np.uint8)
test_split_x = np.memmap(os.path.join(data_path, "test_split_x.npy"), mode="r", shape=(test_val_tiles, 256, 256, 5),
                         dtype=np.uint8)
test_split_y = np.memmap(os.path.join(data_path, "test_split_y.npy"), mode="r", shape=(test_val_tiles, 256, 256),
                         dtype=np.uint8)

train_stats = Mask_Stats(train_split_y)
train_stats.print_stats()
print()
val_stats = Mask_Stats(val_split_y)
val_stats.print_stats()
print()
test_stats = Mask_Stats(test_split_y)
test_stats.print_stats()

Shape: (6672, 256, 256)
Land pixels: 195058814  44.610 %
Valid pixels: 138904480  31.767 %
Invalid pixels: 103292898  23.623 %
Sum: 6672

Shape: (2224, 256, 256)
Land pixels: 65320265  44.816 %
Valid pixels: 46246663  31.730 %
Invalid pixels: 34185136  23.454 %
Sum: 2224

Shape: (2224, 256, 256)
Land pixels: 64786699  44.450 %
Valid pixels: 46892391  32.173 %
Invalid pixels: 34072974  23.377 %
Sum: 2224


In [ ]:
class DataGenerator(Sequence):
    def __init__(self, mmap_x, mmap_y, batch_size):
        self.x_input = mmap_x
        self.y_mask = mmap_y
        self.batch_size = batch_size
        self.num_samples = self.x_input.shape[0]

    # returns number of batches as int
    def __len__(self):
        return int(np.ceil(self.num_samples / float(self.batch_size)))

    # returns single batch
    def __getitem__(self, index):
        batch_indices = slice(index * self.batch_size, (index + 1) * self.batch_size)
        batch_inputs = self.x_input[batch_indices]
        batch_masks = self.y_mask[batch_indices]

        # normalization
        batch_inputs = batch_inputs / 255
        # one-hot-encoding
        batch_masks = np.array([tf.one_hot(item, depth=3).numpy() for item in batch_masks])

        # normalization + one hot encoding
        return batch_inputs, batch_masks

    def getitem_as_img(self, index):
        batch_indices = slice(index * self.batch_size, (index + 1) * self.batch_size)
        batch_inputs = self.x_input[batch_indices]
        batch_masks = self.y_mask[batch_indices]
        # normalization + one hot encoding
        return batch_inputs, batch_masks

In [ ]:
# instanciate DataGenerators
batch_size = 32

train_generator = DataGenerator(train_split_x, train_split_y, batch_size)
val_generator = DataGenerator(val_split_x, val_split_y, batch_size)
test_generator = DataGenerator(test_split_x, test_split_y, batch_size)

print(train_generator.__len__())
print(val_generator.__len__())
print(test_generator.__len__())

209
70
70


In [ ]:
train_batch = train_generator.__getitem__(9)
val_batch = val_generator.__getitem__(3)
test_batch = test_generator.__getitem__(4)


def print_batch_shapes(batch):
    print(type(batch))
    print(batch[0].shape)
    print(batch[1].shape)
    print()


# check batch shapes
print_batch_shapes(train_batch)
print_batch_shapes(val_batch)
print_batch_shapes(test_batch)

# check normalization
print('Check normalization')
print(train_batch[1].max())
print(train_batch[1].min())

print(val_batch[1].max())
print(val_batch[1].min())

print(test_batch[1].max())
print(test_batch[1].min())

print()
# check one-hot-encoding
print('check one hot encoding')
print(train_batch[0].max())
print(train_batch[0].min())

print(val_batch[0].max())
print(val_batch[0].min())

print(test_batch[0].max())
print(test_batch[0].min())

<class 'tuple'>
(32, 256, 256, 5)
(32, 256, 256, 3)

<class 'tuple'>
(32, 256, 256, 5)
(32, 256, 256, 3)

<class 'tuple'>
(32, 256, 256, 5)
(32, 256, 256, 3)

Check normalization
1.0
0.0
1.0
0.0
1.0
0.0

check one hot encoding
1.0
0.0
1.0
0.0
1.0
0.0


### Bug when calculating IoU

In [ ]:
y_pred = np.memmap("../models/experiment_3/predictions/pred_test_0.npy", mode="r", shape=(test_val_tiles, 256, 256, 3),
                   dtype=np.float32)
y_pred = np.argmax(y_pred, axis=-1)
print(y_pred.shape)
print(np.max(y_pred))
print(np.min(y_pred))

(2224, 256, 256)
2
0


In [ ]:
y_pred = to_categorical(y_pred, num_classes=3, dtype="uint8")
print(type(y_pred[0]))
print(y_pred.shape)

<class 'numpy.ndarray'>
(2224, 256, 256, 3)


In [ ]:
y_true = test_split_y
print(type(y_true[0][0][0]))
y_true = to_categorical(y_true, num_classes=3, dtype="uint8")
print(y_true.shape)

<class 'numpy.uint8'>
(2224, 256, 256, 3)


In [ ]:
y_true_0 = y_true[..., 0].flatten()
y_pred_0 = y_pred[..., 0].flatten()
y_true_1 = y_true[..., 1].flatten()
y_pred_1 = y_pred[..., 1].flatten()
y_true_2 = y_true[..., 2].flatten()
y_pred_2 = y_pred[..., 2].flatten()
print(y_true_0.shape)
print(y_pred_0.shape)
print(y_true_1.shape)
print(y_pred_1.shape)
print(y_true_2.shape)
print(y_pred_2.shape)
print(type(y_true_0[0]))
print(type(y_pred_0[0]))
print(type(y_true_1[0]))
print(type(y_pred_1[0]))
print(type(y_true_2[0]))
print(type(y_pred_2[0]))
print(y_pred_0[:5])

(145752064,)
(145752064,)
(145752064,)
(145752064,)
(145752064,)
(145752064,)
<class 'numpy.uint8'>
<class 'numpy.uint8'>
<class 'numpy.uint8'>
<class 'numpy.uint8'>
<class 'numpy.uint8'>
<class 'numpy.uint8'>
[0 0 0 0 0]


In [ ]:
j_invalid = tf.keras.metrics.IoU(num_classes=2, target_class_ids=[0])
j_invalid.update_state(y_true_0, y_pred_0)
print(f'Jaccard invalid_0: {j_invalid.result().numpy()}')

#j_invalid_1 = tf.keras.metrics.IoU(num_classes=2, target_class_ids=[1])
#j_invalid_1.update_state(y_true_0, y_pred_0)
#print(f'Jaccard invalid_1: {j_invalid_1.result().numpy()}')

j_valid = tf.keras.metrics.IoU(num_classes=2, target_class_ids=[0, 1])
j_valid.update_state(y_true_1, y_pred_1)
print(f'Jaccard valid: {j_valid.result().numpy()}')

j_land = tf.keras.metrics.IoU(num_classes=2, target_class_ids=[0, 1])
j_land.update_state(y_true_2, y_pred_2)
print(f'Jaccard land: {j_land.result().numpy()}')

p_invalid = tf.keras.metrics.Precision()
p_invalid.update_state(y_true_0, y_pred_0)
print(f'Precision invalid: {p_invalid.result().numpy()}')

p_valid = tf.keras.metrics.Precision()
p_valid.update_state(y_true_1, y_pred_1)
print(f'Precision valid: {p_valid.result().numpy()}')

p_land = tf.keras.metrics.Precision()
p_land.update_state(y_true_2, y_pred_2)
print(f'Precision land: {p_land.result().numpy()}')



Jaccard invalid_0: 0.6494028568267822
Jaccard invalid_1: 0.6494028568267822
Jaccard valid: 0.791369616985321
Jaccard land: 0.991915762424469
Precision invalid: 0.9689279198646545
Precision valid: 0.9336049556732178
Precision land: 0.9989190697669983


In [ ]:
m_fp = tf.keras.metrics.FalsePositives()
m_fp.update_state(y_true_0, y_pred_0)
fp = m_fp.result().numpy()

m_fn = tf.keras.metrics.FalseNegatives()
m_fn.update_state(y_true_0, y_pred_0)
fn = m_fn.result().numpy()

m_tp = tf.keras.metrics.TruePositives()
m_tp.update_state(y_true_0, y_pred_0)
tp = m_tp.result().numpy()

m_tn = tf.keras.metrics.TrueNegatives()
m_tn.update_state(y_true_0, y_pred_0)
tn = m_tn.result().numpy()

print(f'False Positives: {fp}')
print(f'True Positives: {tp}')
print(f'False Negatives: {fn}')
print(f'True Negatives: {tn}')
print(f'Jaccard invalid: {(tp) / (tp + fn + fp)}')
print(f'Jaccard invalid: {(tn) / (tn + fn + fp)}')
print(f'pixel sum: {tp + fp + tn + fn}')
print(f'correct sum: {2224 * 256 * 256}')
print(f'Precision invalid: {tp / (tp + fp)}')

False Positives: 1222720.0
True Positives: 30766884.0
False Negatives: 3306090.0
True Negatives: 110456368.0
Jaccard invalid: 0.8716893792152405
Jaccard invalid: 0.9606139659881592
pixel sum: 145752064.0
correct sum: 145752064
Precision invalid: 0.9617775678634644


In [ ]:
m_fp = tf.keras.metrics.FalsePositives()
m_fp.update_state(y_true_1, y_pred_1)
fp = m_fp.result().numpy()

m_fn = tf.keras.metrics.FalseNegatives()
m_fn.update_state(y_true_1, y_pred_1)
fn = m_fn.result().numpy()

m_tp = tf.keras.metrics.TruePositives()
m_tp.update_state(y_true_1, y_pred_1)
tp = m_tp.result().numpy()

m_tn = tf.keras.metrics.TrueNegatives()
m_tn.update_state(y_true_1, y_pred_1)
tn = m_tn.result().numpy()

print(f'False Positives: {fp}')
print(f'True Positives: {tp}')
print(f'False Negatives: {fn}')
print(f'True Negatives: {tn}')
print(f'Jaccard valid: {(tp) / (tp + fn + fp)}')
print(f'pixel sum: {tp + fp + tn + fn}')
print(f'correct sum: {2224 * 256 * 256}')
print(f'Precision valid: {tp / (tp + fp)}')

False Positives: 3251528.0
True Positives: 45720908.0
False Negatives: 1171484.0
True Negatives: 95608144.0
Jaccard valid: 0.9117936491966248
pixel sum: 145752064.0
correct sum: 145752064
Precision valid: 0.9336049556732178


In [ ]:
m_fp = tf.keras.metrics.FalsePositives()
m_fp.update_state(y_true_2, y_pred_2)
fp = m_fp.result().numpy()

m_fn = tf.keras.metrics.FalseNegatives()
m_fn.update_state(y_true_2, y_pred_2)
fn = m_fn.result().numpy()

m_tp = tf.keras.metrics.TruePositives()
m_tp.update_state(y_true_2, y_pred_2)
tp = m_tp.result().numpy()

m_tn = tf.keras.metrics.TrueNegatives()
m_tn.update_state(y_true_2, y_pred_2)
tn = m_tn.result().numpy()

print(f'False Positives: {fp}')
print(f'True Positives: {tp}')
print(f'False Negatives: {fn}')
print(f'True Negatives: {tn}')
print(f'Jaccard land: {(tp) / (tp + fn + fp)}')
print(f'pixel sum: {tp + fp + tn + fn}')
print(f'correct sum: {2224 * 256 * 256}')
print(f'Precision land: {tp / (tp + fp)}')

False Positives: 70032.0
True Positives: 64719992.0
False Negatives: 66706.0
True Negatives: 80895328.0
Jaccard land: 0.9978917241096497
pixel sum: 145752048.0
correct sum: 145752064
Precision land: 0.9989190697669983


In [ ]:
m_invalid = tf.keras.metrics.BinaryIoU(target_class_ids=[0])
m_invalid.update_state(y_true_0, y_pred_0)
print(f'Jaccard invalid: {m_invalid.result().numpy()}')

m_invalid_1 = tf.keras.metrics.BinaryIoU(target_class_ids=[1])
m_invalid_1.update_state(y_true_0, y_pred_0)
print(f'Jaccard invalid_1: {m_invalid_1.result().numpy()}')

m_valid = tf.keras.metrics.BinaryIoU(target_class_ids=[0])
m_valid.update_state(y_true_1, y_pred_1)
print(f'Jaccard valid: {m_valid.result().numpy()}')

m_land = tf.keras.metrics.BinaryIoU(target_class_ids=[0])
m_land.update_state(y_true_2, y_pred_2)
print(f'Jaccard land: {m_land.result().numpy()}')


TypeError: ignored

In [ ]:
# precision is always the same, not mather if keras metrics class is used or Trupe positives, .. are calculated
# intersection over union is not the same, even though same input arrays have been used
# sum of confusion matrix values == length of array
# todo calculate with scicit lean